### 원문: https://www.kaggle.com/willkoehrsen/introduction-to-manual-feature-engineering

#### Dataset feature name 정리본: https://chocoffee20.tistory.com/6, https://chocoffee20.tistory.com/8?category=911962 참고

## 이 대회의 목적은? -> 각 고객의 정보를 기반으로 해당 고객이 대출한 돈을 갚을 수 있을지 없을지에 대한 확률을 예측

In [ ]:
#데이터 파일 경로들을 확인합니다.
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
#사용될 라이브러리들을 import합니다.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#경고 문구 무시
import warnings
warnings.filterwarnings('ignore')

#matplotlib 스타일 정의
plt.style.use('fivethirtyeight')

In [ ]:
#bureau.csv 파일을 import합니다.
bureau = pd.read_csv('../input/home-credit-default-risk/bureau.csv')
bureau.head()

#Credit Bureau: 개인신용평가기관 -> bureau.csv: Credit Bureau(CB)에 기록된 타 금융 기관에서 제공받은 모든 고객의 이전 신용 거래

In [ ]:
previous_loan_counts = bureau.groupby('SK_ID_CURR', as_index=False)['SK_ID_BUREAU'].count().rename(columns = {'SK_ID_BUREAU': 'previous_loan_counts'})
previous_loan_counts.head()
#SK_ID_CURR = 고객 ID
#고객 ID를 기준으로 GROUPBY 메서드 실행하여 이전 대출 개수를 파악하고 SK_ID_BUREAU feature의 이름을 previous_loan_counts로 변경

In [ ]:
#application_train.csv에 previous_loan_counts를 merge함. 왼쪽에는 SK_ID_CURR, 오른쪽 끝에는 previous_loan_counts.
train = pd.read_csv('../input/home-credit-default-risk/application_train.csv')
train = train.merge(previous_loan_counts, on='SK_ID_CURR', how='left')

#previous_loan_counts feature의 널값을 0으로 대치
train['previous_loan_counts'] = train['previous_loan_counts'].fillna(0)
train.head()

In [ ]:
#KDE(Kernal Density Estimate, 커널 밀도 추정) -> 단일 변수의 분포를 보여줌. 
def kde_target(var_name, df):
    #var_name: 변수가 되는 feature name
    #df: 대상 dataframe
    
    #새롭게 생성된 변수와 TARGET간의 상관 계수 계산
    corr = df['TARGET'].corr(df[var_name])
    
    #대출 상환 그룹과 상환하지 않은 그룹의 중간값 계산
    avg_repaid = df.loc[df['TARGET']==0, var_name].median()
    avg_not_repaid = df.loc[df['TARGET']==1, var_name].median()
    
    #TARGET값에 따라 그래프 작성
    sns.kdeplot(df.loc[df['TARGET']==0, var_name], label='TARGET == 0')
    sns.kdeplot(df.loc[df['TARGET']==1, var_name], label='TARGET == 1')
    
    #그래프 라벨링, X축은 feature name, Y축은 Density
    plt.xlabel(var_name)
    plt.ylabel('Density')
    plt.title('%s Distribution' % var_name)
    plt.legend()
    
    #상관계수 출력
    print('The correlation between %s and the TARGET is %0.4f' % (var_name, corr))
    
    #중간값 출력
    print('Median value for loan that was not repaid = %0.4f' % avg_not_repaid)
    print('Median value for loan that was repaid =     %0.4f' % avg_repaid)

In [ ]:
#Random Forest 및 Gradient Boosting Machine에 의해 모델 학습에 있어서 중요한 변수로 판명된 EXT_SOURCE_3를 활용하여 테스트. 
#(저자의 이전 노트북에서 실험을 통해 증명되어 있음. 단순 실행)
#EXT_SOURCE_3는 Feature name 중 하나임.
kde_target('EXT_SOURCE_3', train)

In [ ]:
kde_target('previous_loan_counts', train)
#상관계수가 너무 작고, target값에 따른 분포의 차이도 거의 없음 -> 새로운 변수 previous_loan_counts는 중요하지 않음.

In [ ]:
#_agg 접미사는 aggregation의 줄임말. '집계'라는 뜻(ex. count, max, min, ...etc)
bureau_agg = bureau.drop(columns=['SK_ID_BUREAU']).groupby('SK_ID_CURR', as_index=False).agg(['count', 'mean', 'max', 'min', 'sum']).reset_index()
bureau_agg.head()
#고객 ID에 따라 dataframe을 그룹화하고, 대표값(count, mean, max, min, sum)들을 계산.

In [ ]:
#위의 대표값들을 각각 하나의 feature로 분리 후 저장 (ex. ~~~_count, ~~~-mean etc.)
columns = ['SK_ID_CURR']

for var in bureau_agg.columns.levels[0]:
    if var != 'SK_ID_CURR':
        for stat in bureau_agg.columns.levels[1][:-1]:
            columns.append('bureau_%s_%s' %(var, stat))

In [ ]:
#bureau_agg dataframe 살펴보기
bureau_agg.columns = columns
bureau_agg.head()

In [ ]:
train = train.merge(bureau_agg, on='SK_ID_CURR', how='left')
train.head()
#원본 train data와 병합

In [ ]:
#새롭게 생성된 값들과 TARGET과의 상관계수 분석
new_corrs = []

for col in columns:
    corr = train['TARGET'].corr(train[col])
    new_corrs.append((col, corr))

In [ ]:
#상관계수를 절대값에 따라 정렬
new_corrs = sorted(new_corrs, key=lambda x: abs(x[1]), reverse=True)
new_corrs[:15]

In [ ]:
kde_target('bureau_DAYS_CREDIT_mean', train)
#bureau_DAYS_CREDIT -> 고객이 신용관리국 신용등급을 신청한 날로부터 대출신청까지 걸린 기간 = 이전 대출을 받고나서 'Home Credit'에서 대출을 받기 전까지 걸린 일수
#음수값이 크다는 것은 이전 대출이 이루어진 시점이 더 오래됬음을 의미. 
#변수를 많이 만들 때는 주의하자. -> 오버피팅 문제를 일으킬 수 있다.

In [ ]:
#수치데이터의 대표값 연산을 위한 함수 생성
#그냥 위에서 했던 것과 똑같은 작업을 하는 함수를 정의한 것. (이하로 같은 내용임.)
def agg_numeric(df, group_var, df_name):
    #df: 연산의 대상이 되는 dataframe
    #group_var: groupby의 기준이 되는 column(feature name)
    #df_name: column 이름을 재정의하는 데 쓰이는 변수
    
    #agg(출력값): 모든 수치데이터 column들의 대표값이 연산 된 dataframe
    
    #그룹화 대상이 아닌 ID들 제거
    for col in df:
        if col != group_var and 'SK_ID' in col:
            df = df.drop(columns=col)
    
    group_ids = df[group_var]
    numeric_df = df.select_dtypes('number')
    numeric_df[group_var] = group_ids
    
    #특정 변수들을 그룹화 후 해당 대표값들 계산
    agg = numeric_df.groupby(group_var).agg(['count', 'mean', 'max', 'min', 'sum']).reset_index()
    
    #새로운 coulmn 이름 생성
    columns = [group_var]
    
    #변수들에 대해 반복
    for var in agg.columns.levels[0]:
        if var != group_var:
            for stat in agg.columns.levels[1][:-1]:
                columns.append('%s_%s_%s' % (df_name, var, stat))
    
    agg.columns = columns
    
    return agg

In [ ]:
bureau_agg_new = agg_numeric(bureau.drop(columns = ['SK_ID_BUREAU']), group_var = 'SK_ID_CURR', df_name = 'bureau')
bureau_agg_new.head()

In [ ]:
bureau_agg.head()

In [ ]:
#위에서와 마찬가지인 상관계수 계산 함수를 생성
def target_corrs(df):
    corrs = []
    
    for col in df.columns:
        print(col)
        
        if col != 'TARGET':
            corr = df['TARGET'].corr(df[col])
            corrs.append((col, corr))
    
    corrs = sorted(corrs, key=lambda x:abs(x[1]), reverse=True)
    
    return corrs

In [ ]:
#고객의 범주별 대출 갯수를 확인해 count. -> 이들을 정규화 (고객별 대출 횟수의 합계를 1로해 계산)

#dataframe의 범주형 feature들(dtype=='object')에 대해 one-hot encoding 진행
#ex) CREDIT_ACTIVE에는 Active, Bad debt, Closed, Sold 네 가지의 값이 있는데 이를 네 개로 분리해서 0, 1의 값으로 나타낸 것을 알 수 있음.

categorical = pd.get_dummies(bureau.select_dtypes('object'))
categorical['SK_ID_CURR'] = bureau['SK_ID_CURR'] #SK_ID_CURR feature 추가
categorical.head()

In [ ]:
categorical_grouped = categorical.groupby('SK_ID_CURR').agg(['sum', 'mean'])
categorical_grouped.head()

#sum은 고객별 해당 범주에 속한 대출의 총 횟수. mean은 이를 정규화시킨 것

In [ ]:
#.columns.levels[0] -> 가장 위의 name (CREDIT_ACTIVE_active...)
categorical_grouped.columns.levels[0][:10]

In [ ]:
#.columns.levels[1] -> 그 밑의 name (sum, mean)
categorical_grouped.columns.levels[1]

In [ ]:
#위의 표에서 LEVEL을 없애 각각 하나의 FEATURE로 분리 시킨 것.
#LEVEL? -> sum, count로 나눠져있는 층

group_var = 'SK_ID_CURR'

columns = []

for var in categorical_grouped.columns.levels[0]:
    if var != group_var:
        for stat in ['count', 'count_norm']:
            columns.append('%s_%s' % (var, stat))

categorical_grouped.columns = columns

categorical_grouped.head()

In [ ]:
#원래 train에 합병
train = train.merge(categorical_grouped, left_on='SK_ID_CURR', right_index=True, how='left')
train.head()

In [ ]:
train.shape

In [ ]:
#10번째까지의 featur와 123번쨰부터의 feature를 나타낸 것(단순 열람용. 의미 없음.)
train.iloc[:10, 123:]

In [ ]:
#위에서 한 것을 처리하는 함수 생성
def count_categorical(df, group_var, df_name):
    #parameter들은 일전의 parameter와 동일한 느낌
    #categorical(출력값): group_var에 대해 각 고유 범주들의 counts 및 normalized counts 값이 포함된 dataframe
    
    categorical = pd.get_dummies(df.select_dtypes('object'))
    categorical[group_var] = df[group_var]
    categorical = categorical.groupby(group_var).agg(['sum', 'mean'])
    
    column_names = []
    
    for var in categorical.columns.levels[0]:
        for stat in ['count', 'count_norm']:
            column_names.append('%s_%s_%s' % (df_name, var, stat))
    
    categorical.columns = column_names
    
    return categorical

In [ ]:
#함수를 통해 같은 동작 실행. 같은 값이 나옴을 알 수 있다.
bureau_counts = count_categorical(bureau, group_var = 'SK_ID_CURR', df_name = 'bureau')
bureau_counts.head()

In [ ]:
#bureau_balance.csv dataframe 불러오기
#해당 dataframe은 월별 각 고객의 과거 타 금융기관 대출 데이터를 가지고 있음.
#bureau_balance.csv -> 개인신용평가기관에 있는 이전 신용 거래 월 잔액
bureau_balance = pd.read_csv('/kaggle/input/home-credit-default-risk/bureau_balance.csv')
bureau_balance.head()

In [ ]:
# 이하의 내용들은 다음과 같다.
# 각각의 대출에 대해 수치형 대표값들 계산 -> 각각의 대출에 대해 범주형 데이터들의 개수 파악 -> 
# 각각의 대출에 대한 대표값들과 개수를 병합 -> 각 고객별로 이전의 결과에 대한 수치형 대표값들을 계산

#각각의 대출별 상태의 갯수 파악 (이전에 작성한 함수 이용)
bureau_balance_counts = count_categorical(bureau_balance, group_var='SK_ID_BUREAU', df_name='bureau_balance')
bureau_balance_counts.head()

In [ ]:
#각각 'SK_ID_BUREAU'별 대표값들 계산 (이전에 작성한 함수 이용)
bureau_balance_agg = agg_numeric(bureau_balance, group_var='SK_ID_BUREAU', df_name='bureau.balance')
bureau_balance_agg.head()

In [ ]:
#대출을 기준으로 dataframe 그룹화
bureau_by_loan = bureau_balance_agg.merge(bureau_balance_counts, right_index=True, left_on='SK_ID_BUREAU', how='outer')

#SK_ID_CURR을 포함하여 병합
bureau_by_loan = bureau_by_loan.merge(bureau[['SK_ID_BUREAU', 'SK_ID_CURR']], on='SK_ID_BUREAU', how='left')
bureau_by_loan.head()

In [ ]:
bureau_balance_by_client = agg_numeric(bureau_by_loan.drop(columns=['SK_ID_BUREAU']), group_var='SK_ID_CURR', df_name='client')
bureau_balance_by_client.head()

In [ ]:
#새로 짜기 위해 이전 데이터 삭제(위에서 생성한 함수들을 이용해 처음부터 다시 수행함.) -> 생성된 함수들 병합
import gc
gc.enable()

del train, bureau, bureau_balance, bureau_agg, bureau_agg_new, bureau_balance_agg, bureau_balance_counts, bureau_by_loan, bureau_balance_by_client, bureau_counts
gc.collect()

In [ ]:
train = pd.read_csv('../input/home-credit-default-risk/application_train.csv')
bureau = pd.read_csv('../input/home-credit-default-risk/bureau.csv')
bureau_balance = pd.read_csv('../input/home-credit-default-risk/bureau_balance.csv')

In [ ]:
#Bureau dataframe 내 범주형 데이터의 갯수 세기
bureau_counts = count_categorical(bureau, group_var='SK_ID_CURR', df_name='bureau')
bureau_counts.head()

In [ ]:
#Bureau dataframe의 대표값 계산
bureau_agg = agg_numeric(bureau.drop(columns = ['SK_ID_BUREAU']), group_var = 'SK_ID_CURR', df_name = 'bureau')
bureau_agg.head()

In [ ]:
#Bureau Balance dataframe의 각 대출별 범주형 데이터 개수 세기
bureau_balance_counts = count_categorical(bureau_balance, group_var = 'SK_ID_BUREAU', df_name = 'bureau_balance')
bureau_balance_counts.head()

In [ ]:
#Bureau balance dataframe의 각 대출별 대표값 계산
bureau_balance_agg = agg_numeric(bureau_balance, group_var = 'SK_ID_BUREAU', df_name = 'bureau_balance')
bureau_balance_agg.head()

In [ ]:
#bureau balance dataframe의 각 고객별 대표값 계산
bureau_by_loan = bureau_balance_agg.merge(bureau_balance_counts, right_index = True, left_on = 'SK_ID_BUREAU', how = 'outer')
bureau_by_loan = bureau[['SK_ID_BUREAU', 'SK_ID_CURR']].merge(bureau_by_loan, on = 'SK_ID_BUREAU', how = 'left')
bureau_balance_by_client = agg_numeric(bureau_by_loan.drop(columns = ['SK_ID_BUREAU']), group_var = 'SK_ID_CURR', df_name = 'client')

In [ ]:
#원래 train에 있던 column들 리스트 저장(단순 열람용)
original_features = list(train.columns)
print('Original Number of Features: ', len(original_features))

In [ ]:
#bureau counts, bureau_agg, bureau_balance_by_client와 병합
train = train.merge(bureau_counts, on = 'SK_ID_CURR', how = 'left')
train = train.merge(bureau_agg, on = 'SK_ID_CURR', how = 'left')
train = train.merge(bureau_balance_by_client, on = 'SK_ID_CURR', how = 'left')

In [ ]:
#122개에서 333개로 feature 수가 늘어났음을 알 수 있다.
new_features = list(train.columns)
print('Number of features using previous loans from other institutions data: ', len(new_features))

In [ ]:
#누락된 값 확인

def missing_values_table(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(columns= {0: 'Missing Values', 1: '% of total values'})
    mis_val_table_ren_columns = mis_val_table_ren_columns[mis_val_table_ren_columns.iloc[:,1] != 0].sort_values('% of total values', ascending=False).round(1)
    
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
    
    return mis_val_table_ren_columns

In [ ]:
#333개의 feature 중 278개의 feature가 결측값을 가지고 있음을 알 수 있다.
missing_train = missing_values_table(train)
missing_train.head(10)

In [ ]:
#90% 이상의 누락값을 가진 feature제거하고자 함.
missing_train_vars = list(missing_train.index[missing_train['% of total values'] > 90])
len(missing_train_vars)

In [ ]:
#제거하기에 앞서 테스트 데이터에서의 누락된 값의 비율 살펴보기위해 테스트 데이터셋 setting

test = pd.read_csv('../input/home-credit-default-risk/application_test.csv')

test = test.merge(bureau_counts, on='SK_ID_CURR', how='left')
test = test.merge(bureau_agg, on='SK_ID_CURR', how='left')
test = test.merge(bureau_balance_by_client, on='SK_ID_CURR', how='left')

In [ ]:
print('Shape of Testing Data: ', test.shape)

In [ ]:
#train data와 test data가 같은 feature들을(순서를) 가지도록 조정. (align = 두 객체 정렬 method, align 시 target이 없어지기 때문에 미리 저장해놓고 train에 다시 추가
#왜 target이 없어지나? test data에는 target이 없기 때문
train_labels = train['TARGET']
train, test = train.align(test, join='inner', axis=1)
train['TARGET'] = train_labels

In [ ]:
#train, test data 크기 출력
print('Training Data Shape: ', train.shape)
print('Testing Data Shape: ', test.shape)

In [ ]:
#결측치 찾기 수행
missing_test = missing_values_table(test)
missing_test.head(10)

In [ ]:
#test data에서 90%이상의 결측치를 갖는 feature는 몇 개?
missing_test_vars = list(missing_test.index[missing_test['% of total values'] > 90])
len(missing_test_vars)

In [ ]:
missing_columns = list(set(missing_test_vars + missing_train_vars))
print('There are %d columns with more than 90%% missing in either the training or testing data.' % len(missing_columns))

In [ ]:
#90% 이상 결측치를 가지는 feature가 없기 때문에 아무것도 안 지워짐.
train = train.drop(columns = missing_columns)
test = test.drop(columns = missing_columns)

In [ ]:
#중간저장 (없어도 영향 없음)
train.to_csv('train_bureau_raw.csv', index = False)
test.to_csv('test_bureau_raw.csv', index = False)

In [ ]:
#dataframe 상의 모든 상관계수들 계산
corrs = train.corr()

In [ ]:
corrs = corrs.sort_values('TARGET', ascending=False)
#상위 10개 양의 상관계수 (TARGET과의)
pd.DataFrame(corrs['TARGET'].head(10))

In [ ]:
#상위 10개 음의 상관계수
pd.DataFrame(corrs['TARGET'].dropna().tail(10))

In [ ]:
#상관계수가 높다는 것이 그 변수가 유용하다는 것을 의미하지는 않음. 수백개의 변수들을 생성했을 때는 
#random한 noise 때문에 해당 변수들이 상관관계에 있는 것처럼 보일 수도 있음.

#feature들의 유용성을 평가하기 위해 학습된 모델로부터 feature importance들을 살펴보기

#client_bureau_balance_MONTHS_BALANCE_count_mean Feature은 각 고객의 대출별 월별 기록에 대한 평균을 의미함.
#해당 feature는 더 많은 신용 기록을 가지고 있는 고객이 일반적으로 대출금을 상환할 가능성이 높다는 것을 보여줌.
kde_target(var_name='client_bureau_balance_MONTHS_BALANCE_count_mean', df=train)

In [ ]:
kde_target(var_name='bureau_CREDIT_ACTIVE_Active_count_norm', df=train)
#변수가 모든 곳에서 불규칙 -> 상관관계 매우 약함

In [ ]:
#목표값(TARGET)과의 상관계수만 계산하는 것이 아니라, 각 변수간의 상관계수 계산 
#이를 통해 제거해야 할 수도 있는 co-linear한 관계들을 가지는 변수들의 존재 여부를 알 수 있음.
#co-linear한 관계가 왜 좋지 않은가? -> feature들은 독립적이어야 한다(독립변수), 
#co-linear성이 높다는 것은 feature의 불필요한 중복이 있다는 뜻.

#0.8이상의 상관계수 가지는 변수들 찾기 (THRESHOLD:기준점)
threshold = 0.8

above_threshold_vars = {}

for col in corrs:
    above_threshold_vars[col] = list(corrs.index[corrs[col] > threshold])

In [ ]:
#높은 상관관계를 가지는 변수들의 쌍에 대해, 그 쌍 중 하나의 변수들만 제거.

cols_to_remove = []
cols_seen = []
cols_to_remove_pair = []

for key, value in above_threshold_vars.items():
    cols_seen.append(key)
    for x in value:
        if x == key:
            next
        else:
            if x not in cols_seen:
                cols_to_remove.append(x)
                cols_to_remove_pair.append(key)
cols_to_remove = list(set(cols_to_remove))
print('Number of columns to remove: ', len(cols_to_remove))

In [ ]:
#train data와 test data들에서 해당 feature 제거

train_corrs_removed = train.drop(columns=cols_to_remove)
test_corrs_removed = test.drop(columns=cols_to_remove)
print('Training Corrs Removed Shape: ', train_corrs_removed.shape)
print('Testing Corrs Removed Shape: ', test_corrs_removed.shape)

In [ ]:
#중간저장
train_corrs_removed.to_csv('train_bureau_corrs_removed.csv', index = False)
test_corrs_removed.to_csv('test_bureau_corrs_removed.csv', index = False)

# Modeling

In [ ]:
#Model은 Kaggle에서 우수한 성능을 보이는 LightGBM을 사용
import lightgbm as lgb

from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder

import gc

import matplotlib.pyplot as plt

In [ ]:
def model(features, test_features, encoding='one', n_folds=5):
    #features: 모델을 훈련하는데 사용되는 feature들의 dataframe (TARGET 포함)
    #test_features: 모델을 활용해 예측하는데 사용되는 feature들의 dataframe
    #encoding: 범주형 데이터를 인코딩하는 데 사용하는 방식(원핫인코딩->one, integer label encoding->le)
    #n_folds: cross validation에 활용할 fold의 갯수
    
    train_ids = features['SK_ID_CURR']
    test_ids = test_features['SK_ID_CURR']
    
    labels = features['TARGET']
    
    features = features.drop(columns = ['SK_ID_CURR', 'TARGET'])
    test_features = test_features.drop(columns = ['SK_ID_CURR'])
    
    #One hot encoding
    if encoding == 'one':
        features = pd.get_dummies(features)
        test_features = pd.get_dummies(test_features)
        
        #column 일치시키기
        features, test_features = features.align(test_features, join='inner', axis=1)
        
        #기록할 범주형 인덱스 없음
        cat_indices = 'auto'
        
    #Label Encoding: 범주형 feature에 알파벳 순으로 번호를 매기는 것(one-hot처럼 featur수가 늘어나지 않음. 0, 1, 2,...)
    elif encoding == 'le':
        label_encoder = LabelEncoder()
        cat_indices = []
        
        for i, col in enumerate(features):
            if features[col].dtype == 'object':
                #범주형 데이터를 정수로 mapping
                features[col] = label_encoder.fit_transform(np.array(features[col].astype(str)).reshape((-1,)))
                test_features[col] = label_encoder.transform(np.array(test_features[col].astype(str)).reshape((-1,)))
                cat_indices.append(i)
    else:
        raise ValueError('Encoding must be either "ohe" or "le"')
    
    print('Training Data Shape: ', features.shape)
    print('Testing Data Shape: ', test_features.shape)    
    
    feature_names = list(features.columns)
    
    features = np.array(features)
    test_features = np.array(test_features)
    
    #kfold object 생성
    k_fold = KFold(n_splits=n_folds, shuffle=False, random_state=50)
    
    #feature importance 저장하기 위한 배열 생성
    feature_importance_values = np.zeros(len(feature_names))
    
    #예측값 저장하는 array 생성
    test_predictions = np.zeros(test_features.shape[0])
    
    #out of fold validation predictions을 위한 array 생성
    out_of_fold = np.zeros(features.shape[0])
    
    #validation 및 training 점수를 저장하귀 위한 리스트 생성
    valid_scores = []
    train_scores = []
    
    #fold별 반복문 실행
    for train_indices, valid_indices in k_fold.split(features):
        train_features, train_labels = features[train_indices], labels[train_indices]
        valid_features, valid_labels = features[valid_indices], labels[valid_indices]
    
        #모델 생성
        model = lgb.LGBMClassifier(n_estimators=10000, objective = 'binary', 
                                   class_weight = 'balanced', learning_rate = 0.05, 
                                   reg_alpha = 0.1, reg_lambda = 0.1, 
                                   subsample = 0.8, n_jobs = -1, random_state = 50)
    
        #모델 fit
        model.fit(train_features, train_labels, eval_metric = 'auc',
                  eval_set = [(valid_features, valid_labels), (train_features, train_labels)],
                  eval_names = ['valid', 'train'], categorical_feature = cat_indices,
                  early_stopping_rounds = 100, verbose = 200)
    
        #가장 좋았던 iteration 저장
        best_iteration = model.best_iteration_
    
        #feature importance 저장
        feature_importance_values += model.feature_importances_ / k_fold.n_splits
    
        #예측
        test_predictions += model.predict_proba(test_features, num_iteration = best_iteration)[:, 1] / k_fold.n_splits
    
        #out of fold prediction 저장
        out_of_fold[valid_indices] = model.predict_proba(valid_features, num_iteration = best_iteration)[:, 1]
    
        valid_score = model.best_score_['valid']['auc']
        train_score = model.best_score_['train']['auc']
        
        valid_scores.append(valid_score)
        train_scores.append(train_score)
    
        #메모리 초기화
        gc.enable()
        del model, train_features, valid_features
        gc.collect()
    
    #제출용 dataframe 생성
    submission = pd.DataFrame({'SK_ID_CURR': test_ids, 'TARGET': test_predictions})
    
    #feature importance dataframe 생성
    feature_importances = pd.DataFrame({'feature': feature_names, 'importance': feature_importance_values})
    
    valid_auc = roc_auc_score(labels, out_of_fold)
    
    #전체 데이터에 대한 score를 metric에 추가
    valid_scores.append(valid_auc)
    train_scores.append(np.mean(train_scores))
    
    #validation scores를 위한 dataframe 생성
    fold_names = list(range(n_folds))
    fold_names.append('overall')
    
    #training 및 validation score가 저장된 dataframe 생성
    metrics = pd.DataFrame({'fold': fold_names,
                            'train': train_scores,
                            'valid': valid_scores}) 

    return submission, feature_importances, metrics

In [ ]:
#feature importance plot 함수
def plot_feature_importance(df):
    #df: feature importance들이 저장된 dataframe (features/importances)
    df = df.sort_values('importance', ascending = False).reset_index()
    
    #정규화
    df['importance_normalized'] = df['importance'] / df['importance'].sum()
    
    #바 차트 생성
    plt.figure(figsize=(10, 6))
    ax = plt.subplot()
    
    ax.barh(list(reversed(list(df.index[:15]))), df['importance_normalized'].head(15), 
            align = 'center', edgecolor = 'k')
    
    ax.set_yticks(list(reversed(list(df.index[:15]))))
    ax.set_yticklabels(df['feature'].head(15))
    
    plt.xlabel('Normalized Importance'); plt.title('Feature Importances')
    plt.show()
    
    return df

## Feature Engineering이 얼마나 유용했는지 알아보기 (원본 데이터, 적용 데이터, co-linear까지 제거한 데이터)

In [ ]:
#원래 버전 데이터(원본)
train_control = pd.read_csv('../input/home-credit-default-risk/application_train.csv')
test_control = pd.read_csv('../input/home-credit-default-risk/application_test.csv')

In [ ]:
#submission: 제출할 dataframe
#fi: feature importance가 저장된 dataframe
#metrics: validation 및 test 점수가 저장된 dataframe
submission, fi, metrics = model(train_control, test_control)

#training 점수가 validation 점수보다 높다 -> overfit
#submission 점수는 0.745 기록

In [ ]:
fi_sorted = plot_feature_importance(fi)

In [ ]:
metrics

In [ ]:
#feature engineering을 적용한 것
submission_raw, fi_raw, metrics_raw = model(train, test)
#제출물 -> 0.759 (원본의 0.745보다 높다.)

In [ ]:
fi_raw_sorted = plot_feature_importance(fi_raw)

In [ ]:
metrics_raw

In [ ]:
#importance 상위 100개의 feature 중 새로 만든 feature가 얼마나 있을까? -> 51개. feature engineering GOOD!
top_100 = list(fi_raw_sorted['feature'])[:100]
new_features = [x for x in top_100 if x not in list(fi['feature'])]

print('%% of Top 100 Features created from the bureau data = %d.00' % len(new_features))

In [ ]:
#강한 co-linear관계를 가진 feature 제거 데이터
submission_corrs, fi_corrs, metrics_corr = model(train_corrs_removed, test_corrs_removed)
#제출물 -> 0.753 (원본 0.745보다 높지만 단순 feature engineering한 점수인 0.759보다는 낮다.)

In [ ]:
metrics_corr

In [ ]:
fi_corrs_sorted = plot_feature_importance(fi_corrs)

# 자동 피쳐 엔지니어링
https://www.kaggle.com/willkoehrsen/automated-feature-engineering-basics

In [ ]:
#자동 피쳐 엔지니어링? -> 위에서 했던 것과 비슷한 동작을 수행하는 파이썬 라이브러리 featuretools
#데이터베이스에 적용되는 엔티티-관계(ER) 모델에 대한 선행 이해가 필요.
#엔티티와 관계를 정의하면 자동적으로 관계에 맞게 내장 primitives(새로운 계산값, 추후 설명)를 적용해 새로운 feature 생성

import featuretools as ft

#데이터 불러오기 (1000개만)
app_train = pd.read_csv('../input/home-credit-default-risk/application_train.csv').sort_values('SK_ID_CURR').reset_index(drop = True).loc[:1000, :]
app_test = pd.read_csv('../input/home-credit-default-risk/application_test.csv').sort_values('SK_ID_CURR').reset_index(drop = True).loc[:1000, :]
bureau = pd.read_csv('../input/home-credit-default-risk/bureau.csv').sort_values(['SK_ID_CURR', 'SK_ID_BUREAU']).reset_index(drop = True).loc[:1000, :]
bureau_balance = pd.read_csv('../input/home-credit-default-risk/bureau_balance.csv').sort_values('SK_ID_BUREAU').reset_index(drop = True).loc[:1000, :]
cash = pd.read_csv('../input/home-credit-default-risk/POS_CASH_balance.csv').sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index(drop = True).loc[:1000, :]
credit = pd.read_csv('../input/home-credit-default-risk/credit_card_balance.csv').sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index(drop = True).loc[:1000, :]
previous = pd.read_csv('../input/home-credit-default-risk/previous_application.csv').sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index(drop = True).loc[:1000, :]
installments = pd.read_csv('../input/home-credit-default-risk/installments_payments.csv').sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index(drop = True).loc[:1000, :]

In [ ]:
# column
app_train['set'] = 'train'
app_test['set'] = 'test'
app_test["TARGET"] = np.nan

# 데이터프레임에 append (train + test)
app = app_train.append(app_test, ignore_index = True)

In [ ]:
#엔티티 만들기
#엔티티란? 테이블 개체와 비슷하다. 데이터베이스의 엔티티와 같은 의미라고 보면 편하다.
es = ft.EntitySet(id = 'clients')

In [ ]:
#유니크한 index를 가지는 것에 대한 엔티티(중복 값 비허용, 고유ID 등)
es = es.entity_from_dataframe(entity_id = 'app', dataframe = app, index = 'SK_ID_CURR')

es = es.entity_from_dataframe(entity_id = 'bureau', dataframe = bureau, index = 'SK_ID_BUREAU')

es = es.entity_from_dataframe(entity_id = 'previous', dataframe = previous, index = 'SK_ID_PREV')

# 유니크한 index를 가지지 않는 것에 대한 엔티티(중복 값 허용, 보유 자산 등)
es = es.entity_from_dataframe(entity_id = 'bureau_balance', dataframe = bureau_balance, 
                              make_index = True, index = 'bureaubalance_index')

es = es.entity_from_dataframe(entity_id = 'cash', dataframe = cash, 
                              make_index = True, index = 'cash_index')

es = es.entity_from_dataframe(entity_id = 'installments', dataframe = installments,
                              make_index = True, index = 'installments_index')

es = es.entity_from_dataframe(entity_id = 'credit', dataframe = credit,
                              make_index = True, index = 'credit_index')

In [ ]:
#엔티티 간의 관계 정의
r_app_bureau = ft.Relationship(es['app']['SK_ID_CURR'], es['bureau']['SK_ID_CURR'])

r_bureau_balance = ft.Relationship(es['bureau']['SK_ID_BUREAU'], es['bureau_balance']['SK_ID_BUREAU'])

r_app_previous = ft.Relationship(es['app']['SK_ID_CURR'], es['previous']['SK_ID_CURR'])

r_previous_cash = ft.Relationship(es['previous']['SK_ID_PREV'], es['cash']['SK_ID_PREV'])
r_previous_installments = ft.Relationship(es['previous']['SK_ID_PREV'], es['installments']['SK_ID_PREV'])
r_previous_credit = ft.Relationship(es['previous']['SK_ID_PREV'], es['credit']['SK_ID_PREV'])

In [ ]:
#정의된 관계를 엔티티셋에 추가
es = es.add_relationships([r_app_bureau, r_bureau_balance, r_app_previous,
                           r_previous_cash, r_previous_installments, r_previous_credit])
es

In [ ]:
#primitives? 새로운 값 생성하는 방법들. 내장되어 있는 것임.

primitives = ft.list_primitives()
pd.options.display.max_colwidth = 100
primitives[primitives['type'] == 'aggregation'].head(10)

In [ ]:
#featuretools에 내장된 기본 primitives
#primitives는 feature을 생성하기 위해 테이블에 적용되는 작업
default_agg_primitives =  ["sum", "std", "max", "skew", "min", "mean", "count", "percent_true", "num_unique", "mode"]
default_trans_primitives =  ["day", "year", "month", "weekday", "haversine", "num_words", "num_characters"]

# Deep Feature Synthesis
feature_names = ft.dfs(entityset = es, target_entity = 'app',
                       trans_primitives = default_trans_primitives,
                       agg_primitives=default_agg_primitives, max_depth = 2, features_only=True)

print('%d Total Features' % len(feature_names))

In [ ]:
#위에 정의한 primitivies 사용
feature_matrix, feature_names = ft.dfs(entityset = es, target_entity = 'app',
                                       trans_primitives = default_trans_primitives,
                                       agg_primitives=default_agg_primitives, 
                                        max_depth = 2, features_only=False, verbose = True)

pd.options.display.max_columns = 1700
feature_matrix.head(10)

In [ ]:
feature_names[-20:]

In [ ]:
feature_matrix_spec, feature_names_spec = ft.dfs(entityset = es, target_entity = 'app',  
                                                 agg_primitives = ['sum', 'count', 'min', 'max', 'mean', 'mode'], 
                                                 max_depth = 2, features_only = False, verbose = True)

In [ ]:
pd.options.display.max_columns = 1000
feature_matrix_spec.head(10)

In [ ]:
#저자가 미리 작성해 놓은 위의 featuretools를 통해 생성된 feature들의 correlations들을 계산해 놓은 값들이 저장된 csv 파일 사용
correlations = pd.read_csv('../input/home-credit-default-risk-feature-tools/correlations_spec.csv', index_col = 0)
correlations.index.name = 'Variable'
correlations.head()

In [ ]:
correlations_target = correlations.sort_values('TARGET')['TARGET']
# 가장 음의 상관계수를 가지는 feature
correlations_target.head()

In [ ]:
#가장 양의 상관계수를 가지는 feature
correlations_target.dropna().tail()

In [ ]:
#상관 관계를 가지는 변수 분포 시각화
features_sample = pd.read_csv('../input/home-credit-default-risk-feature-tools/feature_matrix.csv', nrows = 20000)
features_sample = features_sample[features_sample['set'] == 'train']
features_sample.head()

In [ ]:
def kde_target_plot(df, feature):
    """Kernel density estimate plot of a feature colored
    by value of the target."""
    
    df = df.reset_index()
    plt.figure(figsize = (10, 6))
    plt.style.use('fivethirtyeight')
    
    sns.kdeplot(df.loc[df['TARGET'] == 0, feature], label = 'target == 0')
    # plot loans that were not repaid
    sns.kdeplot(df.loc[df['TARGET'] == 1, feature], label = 'target == 1')
    
    plt.title('Distribution of Feature by Target Value')
    plt.xlabel('%s' % feature); plt.ylabel('Density');
    plt.show()

In [ ]:
kde_target_plot(features_sample, feature = 'MAX(previous_app.MEAN(credit.CNT_DRAWINGS_ATM_CURRENT))')
#매우 약한 상관관계를 지니고 있음을 알 수 있다. 구분이 안되는 값이기 때문.

In [ ]:
#위에서 한 것처럼 강한 co-linear성을 가진 feature들도 제거할 필요가 있을 수 있음.
threshold = 0.9

correlated_pairs = {}

for col in correlations:
    above_threshold_vars = [x for x in list(correlations.index[correlations[col] > threshold]) if x != col]
    correlated_pairs[col] = above_threshold_vars

In [ ]:
correlated_pairs['MEAN(credit.AMT_PAYMENT_TOTAL_CURRENT)']

In [ ]:
correlations['MEAN(credit.AMT_PAYMENT_TOTAL_CURRENT)'].sort_values(ascending=False).head()

In [ ]:
plt.plot(features_sample['MEAN(credit.AMT_PAYMENT_TOTAL_CURRENT)'], features_sample['MEAN(previous_app.MEAN(credit.AMT_PAYMENT_CURRENT))'], 'bo')
plt.title('Highly Correlated Features');

#강한 co-linear 성을 띄고 있음. 제거를 할 생각을 해야함.

#이후 해당 notebook의 내용은 위의 feature engineering notebook내용과 동일한 흐름으로 진행되기에 기술하지 않음.
#feature importance 확인, feature selection 진행, modeling...